In [22]:
import pandas as pd
import numpy as np

In [2]:
pble = pd.read_csv('../../Dados/PBLE/PBLE.csv', encoding='latin-1', sep=';')

In [3]:
# removendo linha com dados inválidos
pble.drop(27192, inplace=True)

pble['Ano_Ativacao'] = pble['Data de Ativação'].apply(lambda x: int(x[6:]))

In [4]:
pble=pble[pble['Ano_Ativacao']>=2018].sort_values(by='Ano_Ativacao')

In [5]:
pble.head()

,Prestadora,Nº INEP,UF,Municipio,Situação,Tipo de Obrigação,Velocidade de Acesso Instalada,Tecnologia,Nome da Escola,Tipo Escola,Data de Ativação,Mes,Ano,Ano_Ativacao
51271,TELEFÔNICA,35205187[*],SP,Campinas,Instalada,PBLE ...,10 Mbps,ADSL,JOAO VIALTA CEI,Municipal,15/10/2018,3,2022,2018
19850,TELEFÔNICA,35910284,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,SERGIO PAULO MUNIZ PIMENTA CAPITAO,Estadual,22/03/2018,3,2022,2018
19855,TELEFÔNICA,35037497,SP,Guarulhos,Instalada,PBLE ...,2 Mbps,ADSL,BENEDITA DE OLIVEIRA ALE PROFESSORA,Estadual,23/02/2018,3,2022,2018
19872,TELEFÔNICA,35003463,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,REINALDO RIBEIRO DA SILVA DOUTOR,Estadual,19/02/2018,3,2022,2018
19966,TELEFÔNICA,35565817[*],SP,Caraguatatuba,Instalada,PBLE ...,8 Mbps,Rede Celular,LUIZ SILVAR DO PRADO PROFESSOR EMEF,Municipal,05/09/2018,3,2022,2018


In [11]:
pble['velocidade'] = pble['Velocidade de Acesso Instalada'].apply(lambda x: float(x.split(' ')[0]))

In [16]:
pble[['Velocidade de Acesso Instalada', 'velocidade']]

,Velocidade de Acesso Instalada,velocidade
51271,10 Mbps,10.0
19850,2 Mbps,2.0
19855,2 Mbps,2.0
19872,2 Mbps,2.0
19966,8 Mbps,8.0
...,...,...
61705,2 Mbps,2.0
45140,25 Mbps,25.0
40548,25 Mbps,25.0
33122,25 Mbps,25.0


In [48]:
pble_agg_tipo_escola = pble.groupby(['Ano_Ativacao','Municipio','Tipo Escola']).agg(velocidade_median=('velocidade', np.median), velocidade_mean=('velocidade', np.mean)).reset_index()

In [49]:
pble_agg_total_municipio = pble.groupby(['Ano_Ativacao','Municipio']).agg(velocidade_median=('velocidade', np.median), velocidade_mean=('velocidade', np.mean)).reset_index()

### Salvando agregações

In [54]:
pble_agg_total_municipio.to_csv('../../Resultados/Agg/PBLE/PBLE_agg_total_municipio.csv', index=False)

In [55]:
pble_agg_tipo_escola.to_csv('../../Resultados/Agg/PBLE/PBLE_agg_tipo_escola.csv', index=False)